<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    {
        'x1': [0,0,1,0,1,1,0],
        'x2': [0,1,0,1,0,1,0], 
        'x3': [1,1,1,0,0,1,0], 
        'y': [0,1,1,1,1,0,0]
    }
)
X = df.iloc[:,:-1].to_numpy()
y = df.iloc[:,-1:].to_numpy()

In [3]:
# updated Perceptron
import numpy as np

class Perceptron(object):
    
    def __init__(self, inputs=3, hiddenNodes=4, outputNodes=1):
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        # Initialize Weights, randomly
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        sigx = self.__sigmoid(x)
        return sigx * (1 - sigx)

    def predict(self, X):
        # First Layer
        # apply weights to input data
        self.weighted_data = np.dot(X, self.weights1)
        # activate weighted data with sigmoid curve
        self.activated_data = self.__sigmoid(self.weighted_data)
        
        # Second Layer
        # apply weights to first layer data
        self.weighted_output = np.dot(self.activated_data, self.weights2)
        # activate weighted output with sigmoid curve
        self.activated_output = self.__sigmoid(self.weighted_output)
        
        return self.activated_output
    
    def back_correct(self, X, y, output):
        
        # Output Error
        self.output_error = y - output
        
        # if we're calling f(x) = sigmoid(x), then
        # we are acting the output by by f`(x) to get the gradient direction we ought to go in
        # and multiplying by error to get how much we should be moving in that direction
        # remember that 'delta' is the mathematical symbol for the gradiant
        self.output_delta = self.output_error * self.__sigmoid_derivative(output)
        
        # not really sure what z2 error is, but we caculate that here
        self.z2_error = np.dot(self.output_delta, self.weights2.T)
        # we then calculate how much of that error is explained by the inputs
        self.z2_delta = self.z2_error * self.__sigmoid_derivative(self.activated_data)
        
        # finally adjust the weights
        # first set of weights
        self.weights1 += np.dot(X.T, self.z2_delta)
        # second set of weights
        self.weights2 += np.dot(self.activated_data.T, self.output_delta)
    
    def train(self, X, y, niter=10):
        for _ in range(niter):
            output = self.predict(X)
            self.back_correct(X, y, output)


In [5]:
perc = Perceptron()

In [6]:
perc.train(X, y, niter=10_000)

In [7]:
perc.weights1

array([[  1.64016852,   1.52289115, -24.69547171,  27.68351419],
       [  1.75612818,   1.64342071,  26.43434686, -25.93569782],
       [ -0.30753853,  -0.18906347,  -0.22677966,  -0.22691635]])

In [8]:
perc.weights2

array([[ 21.2504773 ],
       [ 14.26059543],
       [-23.91756109],
       [-23.2650546 ]])

In [9]:
# just curious if the weights have an interaction that can be interpret


# definitely does not have meaning that I can interpret
np.dot(perc.weights1, perc.weights2)

array([[ 3.16868238],
       [31.90504   ],
       [ 1.47173941]])

In [10]:
perc.predict(X)

array([[0.00421662],
       [0.99224235],
       [0.99213004],
       [0.99787871],
       [0.99805565],
       [0.00819228],
       [0.00291266]])

In [11]:
# sanity check
xor = pd.DataFrame(
    {
        "x1": [0, 1, 0, 1, 0, 1],
        "x2": [0, 0, 1, 1, 0, 0],
        "noise": [1, 1, 1, 1, 1, 1],
        "y": [0, 1, 1, 0, 0, 1]
    }
)

In [12]:
test_X = xor.iloc[:, :-1].to_numpy()
test_y = xor.iloc[:, -1:].to_numpy()

In [13]:
print("Prediction: \n", perc.predict(test_X), "\n\n Actual Output \n", test_y)

Prediction: 
 [[0.00421662]
 [0.99213004]
 [0.99224235]
 [0.00819228]
 [0.00421662]
 [0.99213004]] 

 Actual Output 
 [[0]
 [1]
 [1]
 [0]
 [0]
 [1]]


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [14]:
import numpy as np
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

//anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/envs/tensorflow_env/lib/python3.6/site-pack

In [15]:
# input image dimensions
img_rows, img_cols = 28, 28

In [16]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [17]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [18]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [19]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [20]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

In [21]:
x_train.shape, y_train.shape

((60000, 784), (60000,))

In [22]:
# trim the training set even further
x_train_sm = x_train[:5_000, :].copy()
y_train_sm = y_train[:5_000].copy()

In [23]:
x_train_sm.shape[1]

784

In [24]:
x_train_sm.shape, x_train.shape, y_train_sm.shape, y_train.shape

((5000, 784), (60000, 784), (5000,), (60000,))

### Estimating Your `net

In [53]:
tron = Perceptron(inputs=784, hiddenNodes=1, outputNodes=5000)
tron.train(x_train_sm, y_train_sm, niter=1)

In [54]:
tron.weights1.shape, tron.weights2.shape

((784, 1), (1, 5000))

In [55]:
tron.predict(x_train_sm)

//anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()


array([[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       ...,
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]])

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?